In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

from HelmholtzSolver3D import *
from ExampleBoundaries import sphere

frequency = 20.0 # frequency [Hz]
k = frequency_to_wavenumber(frequency)

interior_points = np.array([[0.500, 0.000, 0.000],
                           [0.000, 0.000, 0.010],
                           [0.000, 0.000, 0.250],
                           [0.000, 0.000, 0.500],
                           [0.000, 0.000, 0.750]], dtype=np.float32)

# Test Problem 1
# Dirichlet boundary condition with phi = sin(k*z)
#
solver = InteriorHelmholtzSolver3D(sphere())

boundary_condition = solver.dirichlet_boundary_condition()
boundary_condition.f[:] = np.sin(k * solver.centers[:,2])

boundary_incidence = BoundaryIncidence(solver.len())
boundary_incidence.phi.fill(0.0)
boundary_incidence.v.fill(0.0)

interior_incident_phi = np.zeros(interior_points.shape[0], dtype=np.complex64)

boundary_solution = solver.solve_boundary(k, boundary_condition, boundary_incidence)
sample_solution = boundary_solution.solve_samples(interior_incident_phi, interior_points)

print("Test Problem 1")
print("==============\n")
print(boundary_solution)
print(sample_solution)


# Test Problem 2
# Neumann boundary condition with v = cos(k*z)
#
boundary_condition = solver.neumann_boundary_condition()

for i in range(solver.len()):
    a, b, c = solver.geometry.triangle_vertices(i)
    normal = normal_3d(a, b, c)
    boundary_condition.f[i] = k * np.cos(k * solver.centers[i,2]) * normal[2]

boundary_solution = solver.solve_boundary(k, boundary_condition, boundary_incidence)
sample_solution = boundary_solution.solve_samples(interior_incident_phi, interior_points)

print("\n\nTest Problem 2")
print("==============\n")
print(boundary_solution)
print(sample_solution)


# Test Problem 3
# Neumann boundary condition with v = cos(k*z)
#
boundary_condition = solver.neumann_boundary_condition()

p = np.array([0.0, 0.0, 0.0], dtype=np.float32)
for i in range(solver.len()):
    q = solver.centers[i, :]
    r = p - q
    R = norm(r)
    boundary_incidence.phi[i] = np.exp(1.0j * k * R) / (4.0 * np.pi * R)
    a, b, c = solver.geometry.triangle_vertices(i)
    drbdn = -np.dot(r, normal) / R
    boundary_incidence.v[i] = drbdn * np.exp(1.0j * k * R)*(1.0j * k * R - 1.0) / \
            (4.0 * np.pi * R * R)

for i in range(interior_points.shape[0]):
    q = interior_points[i, :]
    r = p - q
    R = norm(r)
    interior_incident_phi[i] = np.exp(1.0j * k * R) / (4.0 * np.pi * R)

boundary_solution = solver.solve_boundary(k, boundary_condition, boundary_incidence)
sample_solution = boundary_solution.solve_samples(interior_incident_phi, interior_points)

print("==============\n")
print(boundary_solution)
print(sample_solution)

Copyright (C) 2017&ndash;2019 Frank Jargstorff

This file is part of the AcousticBEM library.

AcousticBEM is free software: you can redistribute it and/or modify it under the terms of the GNU General Public License as published by the Free Software Foundation, either version 3 of the License, or (at your option) any later version.

AcousticBEM is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for more details.

You should have received a copy of the GNU General Public License along with AcousticBEM. If not, see http://www.gnu.org/licenses/.